In [3]:
import geopandas as gpd
import pydeck as pdk

total_utility_costs = gpd.read_file("total_utility_costs.geojson")

In [4]:
test_data = total_utility_costs.copy()
test_data = test_data.rename({'Index of Multiple Deprivation (IMD) Rank (where 1 is most deprived)':'rank'},axis='columns')
test_data = test_data.sort_values('elec_mean_consumption',ascending=False)
test_data = test_data.head(100)
test_data = test_data.to_crs(4326)

test_data['total_diff_format'] = test_data['total_diff'].apply(lambda x: "£{:.0f}".format((x)))
test_data['elec_mean_consumption_format'] = test_data['elec_mean_consumption'].apply(lambda x: "{:.0f}".format((x)))
test_data['gas_mean_consumption_format'] = test_data['gas_mean_consumption'].apply(lambda x: "{:.0f}".format((x)))


sf_initial_view = pdk.ViewState(latitude=52.4303, longitude=-1.353543, zoom=6, pitch=60)

hx_layer = pdk.Layer(
    'GeoJsonLayer',
    data = test_data ,
    get_elevation='total_diff',
    elevation_scale=1,
    auto_highlight=True,
    pickable=True,
    opacity=0.8,
    get_fill_color='[255, 0, 0]',
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_line_color=[255, 255, 255],
    tooltip=True,
)


tooltip = {
   "html": "<b>Price Increase:</b> {total_diff_format} <br/> <b>Area:</b> {LSOA name (2011)} <br/> <b>Deprivation Rank :</b> {rank} <br/> <b>Gas Mean Consumption :</b> {gas_mean_consumption_format} kWh <br/> <b>Electricity Mean Consumption:</b> {elec_mean_consumption_format} kWh",
   "style": {
        "backgroundColor": "steelblue",
        "color": "white"
   }
}

r = pdk.Deck(layers=[hx_layer], initial_view_state=sf_initial_view, tooltip=tooltip)
r.to_html()